In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## The forward and backward passes

In [2]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

def normalize(x, mean, std): 
    return (x-mean)/std

In [3]:
x_train, y_train, x_valid, y_valid = get_data()

In [4]:
x_train
x_train.shape

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

torch.Size([50000, 784])

In [5]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [9]:
type(train_mean)
train_mean.shape

torch.Tensor

torch.Size([])

In [10]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [11]:
# after normalization, mean is 0 and std is 1
norm_train_mean, norm_train_std = x_train.mean(), x_train.std()
norm_train_mean, norm_train_std

(tensor(0.0001), tensor(1.))

In [12]:
#export
def test_near_zero(a, tol=1e-3): 
    assert a.abs() < tol 
    print(f"Near zero: {a}")

In [13]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

Near zero: 0.00012300178059376776
Near zero: 0.0


In [17]:
# training set has 50000 images, every image is 28x28=784 pixels, images are numbers from 1 to 9
n, m = x_train.shape
c = y_train.max() + 1
cu = y_train.unique()
n, m, c, cu

(50000, 784, tensor(10), tensor([8, 7, 6, 3, 2, 9, 1, 4, 0, 5]))

## Foundations version

### Basic architecture

In [12]:
# torch.randn(m, n) give a matrix with 0 mean and deviation of 1

In [61]:
# num hidden. so input layer=784, hidden layer=50, output layer=1, 
# why output is not 10, probability of 0-9? just a simple demonstrations here.
# in real life, output will be 10, cross entropy to decide the number with the highest probability
nh = 50

In [62]:
# simplified kaiming init / he init
w1 = torch.randn(m, nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [65]:
w1.shape, w1[0]
b1.shape, b1
w2.shape, w2[:5]
b2.shape, b2
w1.mean()
w1.std()
1/math.sqrt(m)

(torch.Size([784, 50]),
 tensor([-0.0043, -0.0712, -0.0534,  0.0497,  0.0389, -0.0226,  0.0006, -0.0075,
          0.0037,  0.0340,  0.0571, -0.0036, -0.0277,  0.0218, -0.0457,  0.0116,
          0.0277,  0.0450,  0.0414, -0.0777,  0.0574, -0.0174, -0.0171,  0.0062,
          0.0021, -0.0366,  0.0491,  0.0019, -0.0125,  0.0449, -0.0195, -0.0133,
          0.0054,  0.0056, -0.0484, -0.0072, -0.1105,  0.0540,  0.0401, -0.0388,
         -0.0314,  0.0072,  0.0014, -0.0391,  0.0157,  0.0127,  0.0535,  0.0057,
         -0.0138,  0.0155]))

(torch.Size([50]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]))

(torch.Size([50, 1]), tensor([[ 0.0703],
         [ 0.0982],
         [ 0.1289],
         [ 0.1922],
         [-0.1006]]))

(torch.Size([1]), tensor([0.]))

tensor(9.7093e-06)

tensor(0.0356)

0.03571428571428571

### why do we want the mean of weight is 0 and standard deviation is $\frac{1} {\sqrt{m}}$? because we used kaiming initialization. dividing the random numbers by square root of m.
in fact, we have an activation function as well. so we really want is the std of activation is about 1. we intialzied with   $\frac{2} {\sqrt{m}}$

In [66]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))


Near zero: 9.709310688776895e-06
Near zero: -9.001418948173523e-05


In [67]:
# This should be ~ (0,1) (mean,std)... because we normalized x_valid with train_mean and train_std
x_valid.mean(), x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [69]:
def lin(x, w, b): 
    return x@w + b

In [70]:
x_valid.shape
w1.shape
t = lin(x_valid, w1, b1)
t.shape


torch.Size([10000, 784])

torch.Size([784, 50])

torch.Size([10000, 50])

In [62]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(), t.std()

(tensor(0.0843), tensor(1.0086))

In [74]:
def relu(x): 
    return x.clamp_min(0.)

In [64]:
t = relu(lin(x_valid, w1, b1))

In [65]:
#...actually it really should be this! after Relu, we removed all negative values, so mean will not be 0, std should about halved.
t.mean(), t.std()

(tensor(0.4423), tensor(0.6341))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

In [66]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [67]:
# std of w1 is not 1 any more
w1.mean(), w1.std()

(tensor(-5.8178e-05), tensor(0.0507))

In [68]:
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.5374), tensor(0.8135))

In [72]:
#export
from torch.nn import init

preserves the magnitude of the variance of the weights when initializing

In [75]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

tensor([[ 0.0575,  0.0385, -0.0167,  ...,  0.0160, -0.0484, -0.0352],
        [ 0.0029, -0.0057, -0.0413,  ..., -0.0764,  0.1001,  0.0823],
        [-0.0260, -0.0501, -0.0353,  ...,  0.0356,  0.0199, -0.0071],
        ...,
        [-0.0097, -0.0223, -0.0458,  ..., -0.0251, -0.0290,  0.0551],
        [ 0.0102, -0.0073, -0.0809,  ..., -0.0310, -0.0123,  0.0066],
        [ 0.0118,  0.0997, -0.0414,  ..., -0.0337,  0.0294,  0.0247]])

In [33]:
init.kaiming_normal_??

$   \text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$

Also known as He initialization.

In [76]:
w1.mean(), w1.std()

(tensor(-0.0003), tensor(0.0508))

In [79]:
t.shape, t.mean(), t.std()

(torch.Size([10000, 50]), tensor(0.5877), tensor(0.8654))

In [81]:
import torch.nn

In [82]:
torch.nn.Linear(m, nh).weight.shape
torch.nn.Linear(m, nh).weight

torch.Size([50, 784])

Parameter containing:
tensor([[-0.0221, -0.0353,  0.0287,  ...,  0.0304, -0.0003, -0.0117],
        [ 0.0310,  0.0343, -0.0128,  ..., -0.0178,  0.0080, -0.0305],
        [ 0.0106,  0.0184, -0.0121,  ...,  0.0129, -0.0202, -0.0214],
        ...,
        [-0.0093, -0.0337, -0.0029,  ..., -0.0315, -0.0005,  0.0287],
        [-0.0280, -0.0109, -0.0077,  ..., -0.0208, -0.0264, -0.0218],
        [ 0.0153,  0.0325,  0.0155,  ...,  0.0109, -0.0168, -0.0078]],
       requires_grad=True)

In [ ]:
torch.nn.Linear.forward??

In [ ]:
torch.nn.functional.linear??

In [ ]:
torch.nn.Conv2d??

In [ ]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [83]:
# what if...?
def relu(x): 
    return x.clamp_min(0.) - 0.5

In [87]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(), t1.std()

(tensor(0.0312), tensor(0.7818))

In [89]:
w1.mean(), w1.std()

(tensor(-0.0003), tensor(0.0507))

In [90]:
def model(xb):
    # xb is the input matrix, e.g. num_images X pixels
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [91]:
model(x_valid).shape

torch.Size([10000, 1])

In [92]:
%timeit -n 10 _ = model(x_valid)

6.31 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [93]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0],1])

In [179]:
# the above all makes sense

### Loss function: MSE

In [97]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon (crossentropy loss). We'll use `mse` for now to keep things simple.)

In [105]:
a = torch.tensor([[1,2,3]])
b = torch.tensor([[1], [2], [3]])
a.shape
a.squeeze()
a.squeeze().shape
b.squeeze()
b.squeeze().shape

torch.Size([1, 3])

tensor([1, 2, 3])

torch.Size([3])

tensor([1, 2, 3])

torch.Size([3])

In [98]:
#export
def mse(output, targ): 
    return (output.squeeze(-1) - targ).pow(2).mean()

In [106]:
y_train, y_valid = y_train.float(), y_valid.float()

In [107]:
# prediction is the output layer, not the mse
preds = model(x_train)

In [108]:
preds.shape

torch.Size([50000, 1])

In [110]:
preds.shape
preds[:2]
preds.min()
preds.max()
preds.squeeze(-1).shape
preds.squeeze(-1)[:2]

torch.Size([50000, 1])

tensor([[0.0262],
        [0.2586]])

tensor(-3.1776)

tensor(2.3653)

torch.Size([50000])

tensor([0.0262, 0.2586])

In [112]:
# model has not been trained yet
mse(preds, y_train)

tensor(31.3875)

In [113]:
preds.shape, preds
y_train

(torch.Size([50000, 1]), tensor([[ 0.0262],
         [ 0.2586],
         [ 0.4051],
         ...,
         [ 0.4289],
         [-0.2489],
         [-1.4365]]))

tensor([5., 0., 4.,  ..., 8., 4., 8.])

all above make sense, forward pass is simple and straight forward.

### Gradients and backward pass

$$\operatorname{MSE}=\frac{1}{n}\sum_{i=1}^n(Y_i-\hat{Y_i})^2$$

* we need find 3 gradients: gradient with respect to output of previous layer, so that we can back pass the gradient
* gradient with respect to weight, and gradient with respect to bias

In [91]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    # in this case the inp is the 50000 predictions, so the gradient of loss is with respect to each of the predictions.
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]
    print('mse grad', inp.g.shape, inp.g, inp.shape, inp, targ.shape, targ)

#### no magical here, just the prediction error times 2 and then divided by total number of predictions, so the gradient is for each of the 50000 predictions. mse_grad returns 50000 grads.

In [106]:
mse_grad(preds, y_train)
# the gradients for the first two predictions are: 
2*(preds[:2].squeeze(-1)-y_train[:2])/50000  
preds.g[:2]
2 * (0.4767 - 5)/50000
2 * (0.0870 - 0)/50000

mse grad torch.Size([50000, 1]) tensor([[-1.5327e-04],
        [ 2.4920e-05],
        [-2.7190e-04],
        ...,
        [-2.9282e-04],
        [-1.5750e-04],
        [-3.0212e-04]]) torch.Size([50000, 1]) tensor([[ 1.1683],
        [ 0.6230],
        [-2.7974],
        ...,
        [ 0.6794],
        [ 0.0625],
        [ 0.4470]]) torch.Size([50000]) tensor([5., 0., 4.,  ..., 8., 4., 8.])


tensor([-1.5327e-04,  2.4920e-05])

tensor([[-1.5327e-04],
        [ 2.4920e-05]])

-0.000180932

3.4799999999999997e-06

In [107]:
# the gradient is store in the g attribute
preds.g

tensor([[-1.5327e-04],
        [ 2.4920e-05],
        [-2.7190e-04],
        ...,
        [-2.9282e-04],
        [-1.5750e-04],
        [-3.0212e-04]])

In [115]:
# use this to see how relu_grad works
def model(xb):
    # xb is the input matrix, e.g. num_images X pixels
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l1

In [123]:
l1 = lin(x_train, w1, b1)
l1.shape
# l1 is relu input
l1[:2]>0

torch.Size([50000, 50])

tensor([[0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
         0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
         0, 1],
        [0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
         0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
         0, 1]], dtype=torch.uint8)

In [108]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    # this out.g is the gradient of mse inp.g
#     print('relu gradient')
#     print('out is:', out.shape, out)
#     print('out.g is:', out.g.shape, out.g)
    # directly pass the gradients to inp from out for all positive inputs, negative inputs get 0 gradient
    inp.g = (inp>0).float() * out.g

In [109]:
relu_grad

<function __main__.relu_grad(inp, out)>

In [126]:
def lin_grad(inp, out, w, b):   
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)    
    b.g = out.g.sum(0)

In [155]:
def lin_grad(inp, out, w, b):
    # think the gradient process as between two layers
    # grad of matmul with respect to input
    # out.g is the gradient of next layer, for lin2, out.g is the prediciton gradients calculated by mse_grad
    # pass the gradients from next layer to previous layer by multiplying the gradient for each image and its corresponding weight
    # image1's gradients for all 50 hidden neurons are just image1's gradient from next layer times each individual weight correspnding to each neuron 
    inp.g = out.g @ w.t()
#     print('nnnnn', out.g[0] * w, inp.g[:1])
#     test_near((out.g[0] * w), inp.g[:1].squeeze().unsqueeze(-1))
#     w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0) # why not -1 for out.g
    # element_wise multiplication and then sum over all images, input squeeze in a third axis, but gradients from next
    # layer squeeze in a second dimension, so the weight gradient dimension is input 2nd dim x gradient 3rd dim, 784x50 in this case.
    # gradient for weight connecting first input and first neuron is the sum of first pixel * gradient from first neuron over all images
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
#     print('mmmm', (inp.unsqueeze(-1) * out.g.unsqueeze(1)).shape, w.g.shape)
    # bias gradient is just the sum of next layer's gradidents for all images
    b.g = out.g.sum(0)
#     print('linear layer gradient')
#     print('out is:', out.shape, out)
#     print('w.g:', w.g.shape, w.g, w.shape, w) 
#     print('b.g', b.g.shape, b.g, b.shape, b) 
#     print('inp.g', inp.g.shape, inp.g, inp.shape, inp)

### how do we actually back pass loss
assuming input layer=1000, hidden layer=9, output layer =2, training images=20000
* weight gradient (linear layer): squeeze in a trailing aixs in input, squeeze in a second axis in gradients of next layer, times them together and then sum over all the training examples. back pass the last two layers: (20000, 9, 1)X(20000, 1, 2), so weights are (9,2). it is like the activation of first neuron in hidden layer times the first gradient in the output layer for all images and then sum over them.
* bias gradient: sum over all images for the gradients in the next layer
* previous layer gradient: gradients of next layer times transposed weights. (20000, 2)x(2, 9)=(20000, 9), this is weighted sum of gradients of next layer

In [124]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
#     print('xxxx', l2.shape, l2.unsqueeze(-1).shape, l2[:2])
    out = l2 @ w2 + b2
    
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
#     print('yyy', out.shape, out.g.shape, out.g.unsqueeze(1).shape, out.g[:2])
    lin_grad(l2, out, w2, b2)
#     print('zzzz', w2.g.shape, l2.g)
#     print('yyy', l2.shape, l2.g.shape, l2.g[:2])
    relu_grad(l1, l2)
#     print('zzz', l1.shape, l1[:2], l1.g.shape, l1.g)
    print('yyy', l1.shape, l1.g.shape, l1.g.unsqueeze(1).shape, l1.g[:2], l1.g.unsqueeze(1)[:2])
    print('YYY', inp.shape, inp.unsqueeze(-1).shape, inp.unsqueeze(-1)[:2])
    lin_grad(inp, l1, w1, b1)
    print('zzzz', w1.g.shape, l1.g)

In [127]:
forward_and_backward(x_train, y_train)

mse grad torch.Size([50000, 1]) tensor([[-1.5327e-04],
        [ 2.4920e-05],
        [-2.7190e-04],
        ...,
        [-2.9282e-04],
        [-1.5750e-04],
        [-3.0212e-04]]) torch.Size([50000, 1]) tensor([[ 1.1683],
        [ 0.6230],
        [-2.7974],
        ...,
        [ 0.6794],
        [ 0.0625],
        [ 0.4470]]) torch.Size([50000]) tensor([5., 0., 4.,  ..., 8., 4., 8.])
yyy torch.Size([50000, 50]) torch.Size([50000, 50]) torch.Size([50000, 1, 50]) tensor([[ 0.0000e+00, -2.1644e-05, -0.0000e+00, -0.0000e+00, -5.5032e-06,
         -0.0000e+00, -3.4785e-05, -0.0000e+00,  1.6820e-05,  0.0000e+00,
          1.6316e-07, -0.0000e+00, -0.0000e+00,  1.5277e-05, -0.0000e+00,
         -0.0000e+00,  0.0000e+00, -0.0000e+00,  0.0000e+00,  4.0771e-05,
          0.0000e+00,  1.1902e-05,  0.0000e+00,  6.9874e-06,  0.0000e+00,
          0.0000e+00, -1.5681e-05,  1.7572e-05, -0.0000e+00,  0.0000e+00,
         -2.4188e-05,  0.0000e+00, -0.0000e+00,  3.8277e-07,  0.0000e+00,
         

In [128]:
# element-wise multiplication
a = torch.tensor([[1,2,3], [4,5,6]]).unsqueeze(-1)
b = torch.tensor([[5], [1]]).unsqueeze(1)
# b == torch.tensor([[5], [1]]).unsqueeze(-1)
#a @ b give a error because shape mismatch
a.shape, a
b.shape, b
c = a * b # broadcasting
c.shape, c
c.sum(0)
c.sum(1)
c.sum(-1)

(torch.Size([2, 3, 1]), tensor([[[1],
          [2],
          [3]],
 
         [[4],
          [5],
          [6]]]))

(torch.Size([2, 1, 1]), tensor([[[5]],
 
         [[1]]]))

(torch.Size([2, 3, 1]), tensor([[[ 5],
          [10],
          [15]],
 
         [[ 4],
          [ 5],
          [ 6]]]))

tensor([[ 9],
        [15],
        [21]])

tensor([[30],
        [15]])

tensor([[ 5, 10, 15],
        [ 4,  5,  6]])

In [139]:
x_train.shape
x_train[:10]
y_train[:10]
forward_and_backward(x_train, y_train)

torch.Size([50000, 784])

tensor([[-0.4244, -0.4244, -0.4244,  ..., -0.4244, -0.4244, -0.4244],
        [-0.4244, -0.4244, -0.4244,  ..., -0.4244, -0.4244, -0.4244],
        [-0.4244, -0.4244, -0.4244,  ..., -0.4244, -0.4244, -0.4244],
        ...,
        [-0.4244, -0.4244, -0.4244,  ..., -0.4244, -0.4244, -0.4244],
        [-0.4244, -0.4244, -0.4244,  ..., -0.4244, -0.4244, -0.4244],
        [-0.4244, -0.4244, -0.4244,  ..., -0.4244, -0.4244, -0.4244]])

tensor([5., 0., 4., 1., 9., 2., 1., 3., 1., 4.])

mse grad torch.Size([50000, 1]) tensor([[-1.5327e-04],
        [ 2.4920e-05],
        [-2.7190e-04],
        ...,
        [-2.9282e-04],
        [-1.5750e-04],
        [-3.0212e-04]]) torch.Size([50000, 1]) tensor([[ 1.1683],
        [ 0.6230],
        [-2.7974],
        ...,
        [ 0.6794],
        [ 0.0625],
        [ 0.4470]]) torch.Size([50000]) tensor([5., 0., 4.,  ..., 8., 4., 8.])
yyy torch.Size([50000, 50]) torch.Size([50000, 50]) torch.Size([50000, 1, 50]) tensor([[ 0.0000e+00, -2.1644e-05, -0.0000e+00, -0.0000e+00, -5.5032e-06,
         -0.0000e+00, -3.4785e-05, -0.0000e+00,  1.6820e-05,  0.0000e+00,
          1.6316e-07, -0.0000e+00, -0.0000e+00,  1.5277e-05, -0.0000e+00,
         -0.0000e+00,  0.0000e+00, -0.0000e+00,  0.0000e+00,  4.0771e-05,
          0.0000e+00,  1.1902e-05,  0.0000e+00,  6.9874e-06,  0.0000e+00,
          0.0000e+00, -1.5681e-05,  1.7572e-05, -0.0000e+00,  0.0000e+00,
         -2.4188e-05,  0.0000e+00, -0.0000e+00,  3.8277e-07,  0.0000e+00,
         

In [140]:
w1.g.shape, w1.shape

(torch.Size([784, 50]), torch.Size([784, 50]))

In [141]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [142]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [143]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [144]:
loss = forward(xt2, y_train)

In [145]:
loss.backward()

In [146]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor model

### Layers as classes

In [114]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): 
        self.inp.g = (self.inp>0).float() * self.out.g

In [138]:
xb = x_train[:2]
xb.shape
xb
# Relu is a class, you can call it as a function, it calls __call__
xout = Relu()(xb)
xout.max()
xout.min()


torch.Size([2, 784])

tensor([[-0.4244, -0.4244, -0.4244,  ..., -0.4244, -0.4244, -0.4244],
        [-0.4244, -0.4244, -0.4244,  ..., -0.4244, -0.4244, -0.4244]])

tensor(2.3171)

tensor(-0.5000)

In [139]:
class Lin():
    def __init__(self, w, b): 
        self.w = w
        self.b = b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp @ self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [140]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [141]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: 
            x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): 
            l.backward()

In [151]:
w1.g, b1.g, w2.g, b2.g = [None] * 4
model = Model(w1, b1, w2, b2)

In [152]:
w1.g

In [153]:
%time loss = model(x_train, y_train)

CPU times: user 218 ms, sys: 7.01 ms, total: 225 ms
Wall time: 38.7 ms


In [154]:
loss

tensor(28.5766)

In [155]:
%time model.backward()

CPU times: user 5.39 s, sys: 2.29 s, total: 7.68 s
Wall time: 6.48 s


In [156]:
model.layers

In [157]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()

In [218]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): 
        raise Exception('not implemented')
        
    def backward(self): 
        self.bwd(self.out, *self.args)

In [219]:
class Relu(Module):
    def forward(self, inp): 
        return inp.clamp_min(0.)-0.5
    
    def bwd(self, out, inp): 
        inp.g = (inp > 0).float() * out.g

In [220]:
class Lin(Module):
    def __init__(self, w, b): 
        self.w = w
        self.b = b
        
    def forward(self, inp): 
        return inp @ self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [222]:
class Mse(Module):
    def forward (self, inp, targ): 
        return (inp.squeeze() - targ).pow(2).mean()
    
    def bwd(self, out, inp, targ): 
        inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [223]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: 
            x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): 
            l.backward()

In [224]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [225]:
%time loss = model(x_train, y_train)

CPU times: user 614 ms, sys: 2.25 s, total: 2.86 s
Wall time: 42 ms


In [226]:
%time model.backward()

CPU times: user 4.19 s, sys: 1min 30s, total: 1min 34s
Wall time: 1.31 s


In [227]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

In [229]:
class Lin(Module):
    def __init__(self, w, b): 
        self.w = w
        self.b = b
        
    def forward(self, inp): 
        return inp @ self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [230]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [231]:
%time loss = model(x_train, y_train)

CPU times: user 707 ms, sys: 2.91 s, total: 3.62 s
Wall time: 50.6 ms


In [232]:
%time model.backward()

CPU times: user 3.33 s, sys: 8.32 s, total: 11.6 s
Wall time: 163 ms


In [233]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [158]:
#export
from torch import nn

In [174]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: 
            x = l(x)
            print(x)
        return self.loss(x.squeeze(), targ)

In [175]:
model = Model(m, nh, 1)

In [176]:
%time loss = model(x_train, y_train)

tensor([[-0.0139, -0.7194,  0.1194,  ..., -0.5361, -0.8814, -0.2212],
        [-1.0406, -0.0535,  0.1731,  ..., -0.7719, -0.6613, -0.6865],
        [ 0.0707,  0.3832,  0.1237,  ..., -0.1785, -0.0776, -0.6716],
        ...,
        [ 0.5378, -1.6935,  0.4396,  ..., -0.3225,  0.2675, -0.4148],
        [-0.0819, -0.8078,  0.4252,  ..., -0.3885, -0.6447, -0.3356],
        [ 0.5932, -0.9925,  1.6636,  ..., -0.7242,  0.7382, -0.5686]],
       grad_fn=<AddmmBackward>)
tensor([[0.0000, 0.0000, 0.1194,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1731,  ..., 0.0000, 0.0000, 0.0000],
        [0.0707, 0.3832, 0.1237,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.5378, 0.0000, 0.4396,  ..., 0.0000, 0.2675, 0.0000],
        [0.0000, 0.0000, 0.4252,  ..., 0.0000, 0.0000, 0.0000],
        [0.5932, 0.0000, 1.6636,  ..., 0.0000, 0.7382, 0.0000]],
       grad_fn=<ThresholdBackward0>)
tensor([[-0.2271],
        [-0.3606],
        [-0.2998],
        ...,
        [-0.1948],
        [-

In [179]:
loss

tensor(28.9134, grad_fn=<MeanBackward1>)

In [180]:
%time loss.backward()

CPU times: user 254 ms, sys: 2.28 ms, total: 256 ms
Wall time: 49.2 ms


In [184]:
model.layers

[Linear(in_features=784, out_features=50, bias=True),
 ReLU(),
 Linear(in_features=50, out_features=1, bias=True)]

## Export

In [240]:
!python notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exp/nb_02.py
